# Fibonacci Heaps

Realizado por André Reis - fc58192 no âmbito da disciplina de Desenho e Desenvolvimento de Algoritmos 2024/2025

## Introdução

Este trabalho tem como tema principal Fibonacci Heaps. Fibonacci Heaps é um algoritmo de priority queue com muito bom desempenho, tendo a maioria das suas operações uma complexidade amoritizada O(1). Este algoritmo consiste num conjunto de árvores min-heap (explicado mais a frente). 
Este algoritmo tem a particularidade de facilitar a maioria das funções e deixar o trabalho mais dificil para uma delas. Com esta caracteristica consegue tornar a maioria das suas funções O(1) e apenas uma O(log(n))

### Descrição do trabalho
Irei começar por explicar a sua motivação e história por detrás da sua criação bem com o seu objetivo. De seguida explicarei melhor o que é o algoritmo em si.
Após essas explicações irei descrever a sua estrutura e as suas principais operações bem como o funcionamento destas.
Por fim passarei a uma implementação prática em python do algoritmo e por fim um exercicio sobre uma implementação prática do algoritmo.

## Motivation and historical background

As Fibonacci Heaps foram criadas em 1984 por Michael Fredman e Robert Tarjan, dois engenheiros informáticos norte-americanos. [1][5][6] Este algoritmo foi originalmente criado com o objetivo de melhorar o tempo amortizado do algoritmo de caminho mais curto de Dijkstra de O(mlog(n)) para O(m+nlog(n)). [7] Este algoritmo é também bom para o uso em outros algoritmos de grafos ou shortest path mas o seu foco acaba por ser priority queues.[1][7][8] 

Este algoritmo tem também um melhor desempenho quando comparado a binomial heaps em operações como decrease-key e merge e com o mesmo desempenho nas restantes operações. [9][10] 

O nome Fibonacci é dado devido as regras usadas na junção de árvores e corte de nós nas árvores. Estes números coincidem com a sequencia de Fibonacci e são usados no calculo da sua complexidade, dai originar o nome Fibonacci Heaps.[1] Explicarei este promenor mais em detalhe mais à frente.

## Design of the algorithm


### O que é 
No contexto de algoritmos, uma heap trata-se de uma estrutura de dados em árvore que dado um node "pai" com um valor k, os seus "filhos" terão sempre um valor <=k (max heap) ou >=k (min heap). Os nodes "filhos" desses "filhos" terão sempre de cumprir a mesma regra que o seu node "pai" cumpre, ou seja, ou a árvore será toda max heap ou será toda min heap. Esta propriedade mantém-se valida independente do tamanho da árvore. [2][4]

Uma Fibonacci Heap é uma estrutura de dados que consiste em várias árvores min heap e é usada em operações de priority queue.[1] Podem também ser usadas em grafos como por exemplo no algoritmo Dijkstra, o que foi uma das razões pela sua criação.[1][8]

Segundo o teorema de Fredman-Tarjan (os criadores do algoritmo), começando com uma Fibonacci Heap vazia, qualquer sequencia "m" de operações insert, extract_min e decrease-key com "n" inserts tem complexidade O(m + nlog(n))[7]

### Estrutura
Como já foi referido, Fibonacci Heaps é um conjunto de árvores min heap. Sendo assim a sua estrutura é semelhante a binomial heaps mas bastante mais flexivel sendo até possivel ser um conjunto de árvores de um node só. Isto no entanto não é ideal e eventualmente a árvore é organizada para que se obtenha a complexidade desejada. Esta organização é efetuada pela função extract_min, sendo esta a única que não possui complexidade constante[1][7].

[7]Cada nó contém:
- um pointer para o seu "pai"
- um pointer para um dos filhos
- um pointer para os seus "irmãos" da esquerda e direita
- o número de "filhos" que contém
- se está marcado ou não. (um nó é marcado se um dos seus filhos foi cortado, exceto se este for o nó de origem, falado mais a frente nas operações de decrease-key)[1]
- o valor do próprio nó

Os nós podem também conter outros parametros opcionais necessários para o que se pertende implementar. Estes parametros poderá ser algo como, por exemplo, nome ou dados extra.

Os nós juntos formam a estrutura toda no geral, é apenas necessário que exista uma classe que represente o algoritmo no geral que contenha um apontador para o menor nó e as funções das operações das priority queues. A lista dos nós de origem será feita através dos apontadores de irmãos que cada nó tem bem como o apontador para o menor nó. Desta maneira é também possivel percorrer todos os filhos de um nó contendo apenas o apontador para um deles.

### Operações
[1][7]Com esta estrutura é possivel realizar algumas operações em tempo constante tais como: 
- Find-min: encontrar o minimo é constante pois já temos um apontador para o menor nó de origem que será sempre o menor nó de todos
- Merge: destina-se a juntar duas fibonacci heaps. Esta operação apenas concatena dois grupos de irmãos. Estes dois grupos são os nós de origem das duas fibonacci heaps a serem juntadas. Tem um custo constante amortizado
- Insert: Adiciona um nó à estrutra. Funciona de maneira similar à função merge e é por isso também constante.
- Decrease-key: esta operação diminui o valor de um nó. Existem ações diferentes a tomar dependo do estado da árvore, nós marcados, e valores dos nós. Existem 3 situações possiveis:
    1. Caso o nó diminuido continue a ser maior que o seu pai então tudo permanece igual.
    2. Caso o nó diminuido fique menor que o seu pai e o seu pai é um nó não marcado então: marca-se o pai, retira-se o nó diminuido (e desmarca-o) e todos os seus filhos passando a ser um nó de origem de uma árvore.
    3. Caso o nó diminuido fique menor que o seu pai e o seu pai é um nó marcado então: retira-se o nó diminuido (e desmarca-o) e todos os seus filhos passando a ser um nó de origem de uma árvore. O pai sofre a mesma alteração do seu filho, tornando-se também um nó de origem de uma árvore. Este processo é repetido recorsivamente caso o pai desse nó seja também um nó marcado.

No caso do decrease-key, o facto de ser recursiva levaria a pensar que esta nunca poderia ter custo constante, no entanto, o facto de marcar e desmarcar o mesmo número de nós leva a que esta não cresça em tamanho e obtenha assim um custo constante amortizado. Esta noção é explorada mais à frente na avaliação de complexidade das operações.

[7]Existem mais operações, no entanto, não tem um custo constante amortizado. Estas são:
- Extract_min que tem um custo O(log n) amortizado.
- Delete que usa as funções Decrease-key e Extract_min, tendo por isso o mesmo custo.

A operação Extract_min é a operação que faz o trabalho necessário para que as outras operações tenham um tempo amortizado constante, organizando as árvores. Assim, esta operação é a mais custosa de todas. A operação Delete, como explicado anteriormente, é uma junção do Decrease-key e Extract, devido a isso acaba por ter a mesma complexidade do Extract_min de O(log n).[1][7]

O modo de funcionamento do Extract_min é o seguinte:
1. Escolhe o menor node e isola-o, separando-o dos seus filhos, irmãos e respetivas árvores originarias. Este é o node que será devolvido no final
2. Junta os filhos do nó retirado aos nós de origem originais. A junção destes é agora o novo conjunto de nós de origem
3. Compara os nós origem em termos do número de filhos que cada um tem. Compara também os valores dos nós para determinar o novo nó minimo
4. Quando deteta dois nós com o mesmo número de filhos junta as duas árvores com o menor desses sendo o pai do outro
5. Repete 3 e 4. recursivamente até que todos os nós restantes tenham um número de filhos diferente dos outros.
6. Os nós restantes são o novo conjunto de nós de origem e é devolvido o nó minimo retirado em 1.

De acordo com esta explicação seria espectável que a complexidade desta operação fosse O(n), no entanto isto acaba por não ser o caso. A razão é deve-se a calculos matemáticos complexos que decidi não incluir, mas simplificando, a razão é a mesma que dá nome ao algoritmo.

### Fibonacci
O nome Fibonacci Heaps deve-se a sua relação com a sequência fibonacci que irá limitar o número de filhos que cada nó tem transformando assim funções que percorrem os filhos dos nós em O(log(n)) em vez de O(n).
Esta relação da-se da seguinte forma:
sendo "x" um nó e "d" o seu numero de filhos, então o número de nós na arvore de root "x" >= F(d+2) sendo F a função Fibonacci.
Esta propriadade deve-se às regras de funcionamento da função Extract_min e limita o crescimento exponencial das árvores na Heap [1][7]

## Implementation in Python 

Neste capitulo irei fazer uma implementação em python deste algoritmo. Farei uma estrutura para um nó e para a fibonacci heap no geral. Junto implementarei as operações necessárias descritas anteriormente, bem como algumas funções auxiliares.

### Classe Node
A classe Node serve como base da heap. Cada implementação de um nó tem representações diferentes dependendo do que se pretende resolver mas o seu funcionamento será igual. Esta implementação visa mostrar o Node num ponto geral sem o especializar para nada.

In [1]:
 
'''Classe Node representa um nó de uma heap e da fibonacci heap no seu geral.
    Serve como base da algoritmo no seu total'''
class Node:

    #inicializador requer um valor que representa a sua prioridade na heap. Um número menor implica uma maior prioridade
    def __init__(self, value):
        self.value = value
        self.parent = None
        self.children = None
        self.leftBrother = self
        self.rightBrother = self
        self.numChild = 0
        self.marked = False
        self.name = "NoName" # campo opcional usado no exercicio

    #marcar um node de forma mais legivel
    def mark(self):
        self.marked = True
    
    #desmarcar um node de forma mais legivel
    def unmark(self):
        self.marked = False

    #inserir um outro node como filho do self
    def insert(self, newNode):
        #parent
        newNode.parent = self 

        if(self.children == None):
            self.children = newNode
        else:
            newNode.insertAsBrother(self.children)
        #nChild
        self.numChild +=1

    #auto-remover-se da linked list de irmãos
    def removeFromBrothers(self):
        #liga os seus irmãos um ao outro
        self.leftBrother.rightBrother = self.rightBrother
        self.rightBrother.leftBrother = self.leftBrother
        #remove os seus pointers de irmãos
        self.leftBrother = self
        self.rightBrother = self
        #marca o pai
        if(self.parent != None):
            self.parent.numChild -= 1
            self.parent.mark()

    #inserir-se na linked list de irmãos de outro nó
    def insertAsBrother(self, node):
        self.leftBrother = node.leftBrother
        self.rightBrother = node
        node.leftBrother = self
        self.leftBrother.rightBrother = self

    #juntar duas linked lists de nós de irmãos
    def mergeBrothers(self, node):
        last1 = self.leftBrother
        last2 = node.leftBrother
        self.leftBrother = last2
        last2.rightBrother = self
        node.leftBrother = last1
        last1.rightBrother = node

    
    #Função de print usada nos testes
    def printNode(self):
        print("SELF: ", self.value, " - Name: ", self.name)
        if(self.parent != None):
            print("Parent: ", self.parent.value)
        else:
            print("Parent: None")
        print("Left: ", self.leftBrother.value," - right: ", self.rightBrother.value)
        if(self.children != None):
            print("Child: ", self.children.value)
        else:
            print("Child: None")
    

### Classe Heap
Agora que já temos um nó podemos implementar a fibonacci heap no geral. Esta tem o nome heap mas representa o conjunto das heaps. Possui também as funções para as operações necessárias para uma priority queue descritas no capitulo anterior.

In [8]:
#class Heap representa uma fibonacci heap
''''''
class Heap:

    #Requer um nó inicial
    def __init__(self, min):
        self.min = min
        self.marked = set()
    
    #inserir um novo nó na heap
    def insert(self, new):
        new.insertAsBrother(self.min)
        if(new.value < self.min.value):
            self.min = new

    #juntar duas heaps (simplesmente juntar as suas duas rootlists)
    def merge(self, newHeap):
        self.min.mergeBrothers(newHeap.min)
        if(self.min.value > newHeap.min.value):
            self.min = newHeap.min
    
    #devolve o menor nó
    def findMin(self):
        return self.min

    #diminui o valor de um determinado nó
    def decrease_key(self, node, newValue):

        #função auxiliar do decrease key, arranca um nó do seu pai e leva-o para a root-list 
        def toRoot(heap, node):
            # print("bbbbb")
            # node.printNode()
            loop = False
            parent = None
            if(node.parent != None):
                loop = node.parent.marked
                node.parent.mark()
                parent = node.parent

                #atribui um novo pointer de filho ao pai
                if(node.rightBrother != node):
                    node.parent.children = node.rightBrother
                else:
                    node.parent.children = None


            #remove da lista de irmãos
            node.removeFromBrothers()
            #adiciona a root
            node.insertAsBrother(heap.min) 
            #tirar pai
            node.parent = None
            #unmark
            node.unmark()
            #check novo min
            if(heap.min.value > node.value):
                heap.min = node

            #repete a funcao recursivamente caso o pai do no atual esteja marcado
            if(loop):
                heap = toRoot(heap, parent)

            return heap
        #muda valor
        node.value = newValue
        if(node.parent != None and node.parent.value > newValue):
            
            self = toRoot(self, node)

    #devolve e remove o menor nó. Também reestrutura a heap de acordo com as regras de uma fibonacci heap
    def extract_min(self):

        rootList = self.min
        #checks caso a heap esteja (quase) vazia
        if(self.min.rightBrother != self.min or self.min.leftBrother != self.min):
            rootList = self.min.rightBrother 
            self.min.removeFromBrothers() # separa o min do resto
        elif(self.min.children != None):
            rootList = self.min.children
        else:
            self.min = None
            return rootList
        

        if(self.min.children != None):
            rootList.mergeBrothers(self.min.children) # junta filhos do min com root list original
        new_min = rootList

        #junção de roots de acordo com numero de filhos
        nChildDict = dict()
        while(rootList not in nChildDict.values()): #percorre todos os nós da rootList ate estarem todos no dicionario
            rootList.parent = None
            #caso seja um novo min
            if(rootList.value < new_min.value):
                new_min = rootList
            #caso nenhum outro node tenha o mesmo numero de filhos
            if(rootList.numChild not in nChildDict.keys()):
                nChildDict[rootList.numChild] = rootList
                rootList = rootList.rightBrother
            #caso exista um outro node com o mesmo numero de filhos
            else:
                n = rootList.numChild
                other = nChildDict.get(n)
                #o node maior torna-se filho do node menor
                if(rootList.value < other.value):
                    other.removeFromBrothers()
                    rootList.insert(other)
                    nChildDict.pop(n)
                    nChildDict[rootList.numChild] = rootList
                    rootList = rootList.rightBrother
                else:
                    rootList.removeFromBrothers()
                    other.insert(rootList)
                    nChildDict.pop(n)
                    nChildDict[other.numChild] = other
                    rootList = other.rightBrother
                   
        old_min = self.min
        self.min = new_min

        return old_min

    #apaga um nó tornando-o o menor nó e extraindo-o   
    def delete_node(self, node):
        min_value = self.min.value
        self.decrease_key(node, min_value-1)
        self.extract_min()

    #check se esta vazia
    def isEmpty(self):
        if(self.min == None):
            return True
        else:
            return False
        
    #Funcao de print usada para testes
    def printAll(self):
        def printRec(node, original):
            node.printNode()
            if(node.children != None):
                printRec(node.children, node.children)
            if(node.rightBrother.value == original.value):
                return
            if(node.value != node.rightBrother.value and node.rightBrother.value != original.value):
                printRec(node.rightBrother, original)
                
        
        printRec(self.min, self.min)
        



### Testes
Testes simples usados para testar e provar o funcionamento correto da implementação acima realizada.

In [9]:
#Codigo de testes
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)
n5 = Node(5)
n6 = Node(6)
n7 = Node(7)
n8 = Node(8)
n9 = Node(9)
n10 = Node(10)


heap = Heap(n1)
heap.insert(n2)
heap.insert(n3)
heap.insert(n5)
heap.insert(n4)
heap.insert(n6)

heap.extract_min()
heap.decrease_key(n3, 1)

print("---------------------------------------------------")
heap.printAll()

print("aaaa")
n4.printNode()
heap.delete_node(n4)

print("---------------------------------------------------")
heap.printAll()

heap.extract_min()
print("---------------------------------------------------")
heap.printAll()

---------------------------------------------------
SELF:  1  - Name:  NoName
Parent: None
Left:  6  - right:  2
Child: None
SELF:  2  - Name:  NoName
Parent: None
Left:  1  - right:  6
Child:  4
SELF:  4  - Name:  NoName
Parent:  2
Left:  4  - right:  4
Child:  5
SELF:  5  - Name:  NoName
Parent:  4
Left:  5  - right:  5
Child: None
SELF:  6  - Name:  NoName
Parent: None
Left:  2  - right:  1
Child: None
aaaa
SELF:  4  - Name:  NoName
Parent:  2
Left:  4  - right:  4
Child:  5
---------------------------------------------------
SELF:  1  - Name:  NoName
Parent: None
Left:  1  - right:  1
Child:  6
SELF:  6  - Name:  NoName
Parent:  1
Left:  2  - right:  2
Child: None
SELF:  2  - Name:  NoName
Parent:  1
Left:  6  - right:  6
Child:  5
SELF:  5  - Name:  NoName
Parent:  2
Left:  5  - right:  5
Child: None
---------------------------------------------------
SELF:  2  - Name:  NoName
Parent: None
Left:  6  - right:  6
Child:  5
SELF:  5  - Name:  NoName
Parent:  2
Left:  5  - right:  5
C

## Análise de complexidade

### Classe Node

A classe Node serve como base para o algoritmo num todo. Esta contém as funções:
- mark
- unmark
- insert
- removeFromBrothers
- insertAsBrother
- mergeBrothers

Todas estas funções tem complexidade O(1). Vejamos agora cada uma individualmente.

#### mark
Simplesmente muda uma váriavel, sendo assim O(1).

#### unmark
Igual a "mark", simplesmente muda uma váriavel, sendo assim O(1).

#### insert
Não contém qualquer tipo de loop e limita-se a mudar os vários parametros de diversos nós. O número de nós e parametros alterados é sempre constante e devido a isso esta função é também O(1).

#### removeFromBrothers
Limita-se a alterar as variaveis de pointers de irmãos de 3 nós. Não contém qualquer loop e todas as chamadas da função terão o mesmo tempo. Devido a isso é O(1).

#### insertAsBrother
Esta função faz o oposto de "removeFromBrothers" da mesma forma. Não contém qualquer tipo de loop e é O(1).

#### mergeBrothers
Semelhante às duas funções anteriores, apenas muda sempre as mesmas variaveis e não contém qualquer tipo de loop. É por isso também O(1).


### Classe Heap
A classe Heap contém as funções principais esperadas do algoritmo. Estas são:
- insert
- merge
- findMin
- decrease_key, esta função contém a função "toRoot" 
- extract_min
- delete_node 

A maioria tem complexidade O(1) mas não todas. Vejamos agora a cada uma individualmente.

#### insert
Esta função limita-se a adicionar um node à rootList. Para tal, esta chama a função "insertAsBrothers" da classe Node. Como visto anteriormente esta função tem complexidade O(1). Devido a isso, a função "insert" tem também complexidade O(1).

#### merge
Semelhantemente à função anterior, o merge limita-se a chamar uma função da classe Node, sendo esta a função "mergeBrothers". Esta função é também O(1) o que resulta na função merge ser também O(1).

#### findMin
Esta função é um simples "getter" sendo por isso O(1).

#### decrease-key
Decrease-key é uma função dividida em duas partes. A sua parte final é uma simples atribuição de valores sendo por isso O(1). Por outro lado, a primeira parte é uma chamada à função "toRoot" contida dentro de si. Devido a isto, a complexidade desta função é depende da complexidade de "toRoot"

#####  toRoot
Esta função está também dividida em duas partes. A primeira parte é uma simples atribuição de valores O(1). A segunda parte da função é uma recursão. Esta recursão, no entanto, só irá ocorrer caso o pai do nó em questão esteja marcado. Para que um nó seja marcado será necessário que este perca um filho, e para tal é necessário que se chame a função em questão. 

Um nó que foi cortado e voltou à root perderá a sua marca. Devido a este facto, todas as chamadas desta função, com exceção da primeira, irão marcar um nó e desmarcar outro. Devido a isto, a sua complexidade irá se manter constante (O(1)).[7]

### extract_min
Esta função é a mais complexa de todo o algoritmo e por isso está dividida em diversas partes.
Numa primeira parte é retirado o min e os seus filhos são adicionados à rootList. Esta parte utiliza o "findMin" e o "mergeBrothers" ambas funções da classe Node e ambas com complexidade O(1). Devido a isso, esta primeira parte tem uma complexidade de O(1).

Na segunda parte é necessário percorrer os nós todos pertencentes à rootList e junta-los de acordo com o número de filhos que cada um tem. Para guardar quantos "filhos" cada nó percorrido tem em comparação com outros é usado um dicionário, devido a isso a comparação de nós tem uma complexidade neglegenciavel quando comparado com a complexidade de ter de percorrer todos os nós da rootList.

Com isto, esta parte teria complexidade O(n) onde n é o número de nós na rootList. No entanto, devido às regras relativas ao número de filhos dos nós aplicadas por esta função relacionadas com o crescimento da árvore e a sequência de fibonacci, então esta secção acaba por ter uma complexidade de O(log(n)).

A ultima parte da função destina-se a uma simples mudaça de valores com tempo constante O(1).
Juntando as 3 partes da função conclui-se que esta tem uma complexidade O(log(n)).


## Exercicio relacionado

Nesta secção irei demonstar uma possivel implementação de uso de Fibonacci Heaps para um cenário ficticio com base na realidade.

### Cenário

Um empresa está a realizar um projeto e para tal efetuou uma análise de possiveis riscos que poderiam afetar o projeto. Tendo realizado essa análise a empresa dividiu os riscos em diferentes numeros e tipos para melhor organização. A empresa atribuiu um valor a cada risco representando a sua prioridade no tratamento (menor nº implica uma maior prioridade) de acordo com a sua facilidade de resolver de modo a diminuir custos.

Ao realizar o tratamento de um risco de um certo tipo e numero, outros com o mesmo tipo ou numero tornam-se mais fáceis de resolver e por isso a sua prioridade aumenta (o seu valor diminui).

Qual a melhor ordem para que a empresa trate destes riscos de modo a diminuir os seus custos?

### Objetivo do exercicio

O objectivo deste exercicio é de demonstrar o uso de deste algoritmo numa priority queue usando a sua vantagem em relação aos outros como binomial heaps. Esta vantagem deve-se à complexidade amortizada das operações de merge e decrease-key. Este exercicio não requer o uso da operação merge, no entando, requer a utilização frequente da operação decrease-key.

### Implementação do Cenário

In [ ]:
#Riscos da empresa (Todos os valores foram atribuidos aleatoriamente)

# Tipo 1
risco_1_tipo_1 = Node(10)
risco_1_tipo_1.name = "risco_1_tipo_1"
risco_2_tipo_1 = Node(5)
risco_2_tipo_1.name = "risco_2_tipo_1"
risco_3_tipo_1 = Node(16)
risco_3_tipo_1.name = "risco_3_tipo_1"
risco_4_tipo_1 = Node(7)
risco_4_tipo_1.name = "risco_4_tipo_1"
risco_5_tipo_1 = Node(23)
risco_5_tipo_1.name = "risco_5_tipo_1"

# Tipo 2
risco_1_tipo_2 = Node(24)
risco_1_tipo_2.name = "risco_1_tipo_2"
risco_2_tipo_2 = Node(3)
risco_2_tipo_2.name = "risco_2_tipo_2"
risco_3_tipo_2 = Node(15)
risco_3_tipo_2.name = "risco_3_tipo_2"
risco_4_tipo_2 = Node(19)
risco_4_tipo_2.name = "risco_4_tipo_2"
risco_5_tipo_2 = Node(1)
risco_5_tipo_2.name = "risco_5_tipo_2"

# Tipo 3
risco_1_tipo_3 = Node(6)
risco_1_tipo_3.name = "risco_1_tipo_3"
risco_2_tipo_3 = Node(18)
risco_2_tipo_3.name = "risco_2_tipo_3"
risco_3_tipo_3 = Node(11)
risco_3_tipo_3.name = "risco_3_tipo_3"
risco_4_tipo_3 = Node(9)
risco_4_tipo_3.name = "risco_4_tipo_3"
risco_5_tipo_3 = Node(20)
risco_5_tipo_3.name = "risco_5_tipo_3"

# Tipo 4
risco_1_tipo_4 = Node(17)
risco_1_tipo_4.name = "risco_1_tipo_4"
risco_2_tipo_4 = Node(25)
risco_2_tipo_4.name = "risco_2_tipo_4"
risco_3_tipo_4 = Node(8)
risco_3_tipo_4.name = "risco_3_tipo_4"
risco_4_tipo_4 = Node(2)
risco_4_tipo_4.name = "risco_4_tipo_4"
risco_5_tipo_4 = Node(13)
risco_5_tipo_4.name = "risco_5_tipo_4"

# Tipo 5
risco_1_tipo_5 = Node(14)
risco_1_tipo_5.name = "risco_1_tipo_5"
risco_2_tipo_5 = Node(21)
risco_2_tipo_5.name = "risco_2_tipo_5"
risco_3_tipo_5 = Node(22)
risco_3_tipo_5.name = "risco_3_tipo_5"
risco_4_tipo_5 = Node(4)
risco_4_tipo_5.name = "risco_4_tipo_5"
risco_5_tipo_5 = Node(12)
risco_5_tipo_5.name = "risco_5_tipo_5"



#Lista para tipo
tipo_1 = {risco_1_tipo_1, risco_2_tipo_1, risco_3_tipo_1, risco_4_tipo_1, risco_5_tipo_1}
tipo_2 = {risco_1_tipo_2, risco_2_tipo_2, risco_3_tipo_2, risco_4_tipo_2, risco_5_tipo_2}
tipo_3 = {risco_1_tipo_3, risco_2_tipo_3, risco_3_tipo_3, risco_4_tipo_3, risco_5_tipo_3}
tipo_4 = {risco_1_tipo_4, risco_2_tipo_4, risco_3_tipo_4, risco_4_tipo_4, risco_5_tipo_4}
tipo_5 = {risco_1_tipo_5, risco_2_tipo_5, risco_3_tipo_5, risco_4_tipo_5, risco_5_tipo_5}

tipo_list = [tipo_1, tipo_2, tipo_3, tipo_4, tipo_5]

#Lista para numeros
risco_1 = {risco_1_tipo_1, risco_1_tipo_2, risco_1_tipo_3, risco_1_tipo_4, risco_1_tipo_5}
risco_2 = {risco_2_tipo_1, risco_2_tipo_2, risco_2_tipo_3, risco_2_tipo_4, risco_2_tipo_5}
risco_3 = {risco_3_tipo_1, risco_3_tipo_2, risco_3_tipo_3, risco_3_tipo_4, risco_3_tipo_5}
risco_4 = {risco_4_tipo_1, risco_4_tipo_2, risco_4_tipo_3, risco_4_tipo_4, risco_4_tipo_5}
risco_5 = {risco_5_tipo_1, risco_5_tipo_2, risco_5_tipo_3, risco_5_tipo_4, risco_5_tipo_5}

risco_list = [risco_1, risco_2, risco_3, risco_4, risco_5]


### Solução

In [ ]:

# Uso com a fibonacci heap
fib_heap = Heap(risco_1_tipo_1)

# Preencher a heap:
for i in tipo_list:
    for x in i:
        if(x.name != "risco_1_tipo_1"):
            fib_heap.insert(x)



lista_ordenada = []

# percorre a fibonacci heap toda.
# para cada min encontrado, retira-o adiciona à lista e diminui os valores de todos os outros riscos com o mesmo numero ou tipo
while(not fib_heap.isEmpty()): 

    #remove o minimo, este será o próximo risco a ser tratado
    min = fib_heap.extract_min()
    lista_ordenada.append(min.name)
    print(f"Risco a tratar: {min.name}, Custo: {min.value}")
    
    #diminui o valor de riscos do mesmo tipo
    for i in tipo_list:
        if min in i:
            i.remove(min)
            for j in i:
                fib_heap.decrease_key(j, j.value-1)
            break

    #diminui o valor de riscos com o mesmo numero
    for i in risco_list:
        if min in i:
            i.remove(min)
            for j in i:
                fib_heap.decrease_key(j, j.value-1)
            break


print(lista_ordenada)

Risco a tratar: risco_5_tipo_2, Custo: 1
Risco a tratar: risco_4_tipo_4, Custo: 2
Risco a tratar: risco_2_tipo_2, Custo: 2
Risco a tratar: risco_4_tipo_5, Custo: 3
Risco a tratar: risco_1_tipo_3, Custo: 6
Risco a tratar: risco_2_tipo_1, Custo: 4
Risco a tratar: risco_4_tipo_1, Custo: 4
Risco a tratar: risco_4_tipo_3, Custo: 5
Risco a tratar: risco_3_tipo_4, Custo: 7
Risco a tratar: risco_1_tipo_1, Custo: 7
Risco a tratar: risco_3_tipo_3, Custo: 8
Risco a tratar: risco_5_tipo_5, Custo: 10
Risco a tratar: risco_5_tipo_4, Custo: 9
Risco a tratar: risco_1_tipo_5, Custo: 10
Risco a tratar: risco_3_tipo_2, Custo: 11
Risco a tratar: risco_3_tipo_1, Custo: 10
Risco a tratar: risco_4_tipo_2, Custo: 12
Risco a tratar: risco_1_tipo_4, Custo: 11
Risco a tratar: risco_2_tipo_3, Custo: 13
Risco a tratar: risco_5_tipo_3, Custo: 13
Risco a tratar: risco_2_tipo_5, Custo: 15
Risco a tratar: risco_3_tipo_5, Custo: 14
Risco a tratar: risco_5_tipo_1, Custo: 15
Risco a tratar: risco_1_tipo_2, Custo: 16
Risc

## Referencias:

1. https://en.wikipedia.org/wiki/Fibonacci_heap 
2. https://en.wikipedia.org/wiki/Heap_(data_structure) 
3. https://en.wikipedia.org/wiki/Priority_queue 
4. https://www.geeksforgeeks.org/heap-data-structure/ 
5. https://en.wikipedia.org/wiki/Michael_Fredman
6. https://en.wikipedia.org/wiki/Robert_Tarjan
7. https://www.cs.princeton.edu/~wayne/kleinberg-tardos/pdf/FibonacciHeaps.pdf - Jon Kleinberg and Éva Tardos
8. https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm 
9. https://en.wikipedia.org/wiki/Binomial_heap 
10. https://www.cs.cmu.edu/afs/cs/academic/class/15750-s17/ScribeNotes/lecture3.pdf - Carnegie Mellon University. (2017)
11. https://github.com/danielborowski/fibonacci-heap-python - Daniel Borowski
12. Gerth Stølting Brodal, George Lagogiannis, and Robert E. Tarjan. 2012. Strict fibonacci heaps. In Proceedings of the forty-fourth annual ACM symposium on Theory of computing (STOC '12). Association for Computing Machinery, New York, NY, USA, 1177–1184. https://doi.org/10.1145/2213977.2214082